# Computer Vision Workshop

Dieses Tutorial zeigt, wie man mittels eines Neuronalen Netzwerks / Deep Learning einen Bild-Klassifizierer baut, der Katzenbilder von Hundebildern unterscheidet.

Diese Aufgabenstellung kommt aus dem ["Cats vs. Dogs"](https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition) Wettbewerb der Website Kaggle.

Basis bildet ein Convolutional Neuronal Network (CNN) namens "VGG16", welches auf Basis der Daten des [Imagenet Datasets](http://image-net.org/synset?wnid=n02084071) vortrainiert wurde. Das Modell wird durch Umkonfiguration und Re-Training so angepasst, dass es die gestellte Aufgabe lösen kann.

Die Grundlagen zu diesem Workshop kommen aus dem Deep Learning MOOC [fast.ai](http://fast.ai).

In [ ]:
%matplotlib inline

## Data preparation

Die Daten des Kaggle Wettbewerbs wurden schon vorbereitet und in der "richtigen" Struktur abgelegt.
Das Verzeichnis `data` enthält die Trainings- und Validierungsdaten aus dem Dataset. Dabei sind die Bilder zu jeder zu erkennenden "Klasse" (Cats & Dogs in unserem Fall) in einem eigenen Unterverzeichnis abgelegt:

In [ ]:
# ! führt einen Shell-Befehl aus...
!tree -d data 

Mal schauen, wieviele Dateien in den Trainings- und Validerungsdaten drin sind:

In [ ]:
!echo -n "Training cats: " && ls data/train/cats | wc -w
!echo -n "Training dogs: " && ls data/train/dogs | wc -w
!echo -n "Validation cats: " && ls data/valid/cats | wc -w
!echo -n "Validation dogs: " && ls data/valid/dogs | wc -w

Das Verzeichnis `test` enthält die Bilder, die nicht klassifizert sind (deshalb das Unterverzeichnis `unknown`). Diese wollen wir nach dem Training bestimmen. Mal sehen, wieviele das sind:

In [ ]:
!echo -n "Test: " && ls data/test/unknown | wc -w

Damit wir am Code herumprobieren können, ohne gleich lange Laufzeiten aufgrund der vielen Dateien zu erhalten, gibt es noch ein `sample` Dataset, welches gleich aufgebaut ist, aber nur nur einen kleinen Teil der Daten enthält:

In [ ]:
!tree -d sample

In [ ]:
!echo -n "Training cats: " && ls sample/train/cats | wc -w
!echo -n "Training dogs: " && ls sample/train/dogs | wc -w
!echo -n "Validation cats: " && ls sample/valid/cats | wc -w
!echo -n "Validation dogs: " && ls sample/valid/dogs | wc -w
!echo -n "Test: " && ls sample/test/unknown | wc -w

## Training

In [ ]:
from glob import glob
import numpy as np
import shutil
import os.path

import numpy as np
np.set_printoptions(precision=4, linewidth=100)

Hier setzen wir den Pfad für die Daten, mit denen wir arbeiten wollen (also `data` oder `sample`):

In [ ]:
path = "data/"
# path = "sample/"
path = os.path.join(os.path.curdir,path)
print path

Wir laden die Python Klasse, welche das Modell (ein CNN) in ein nettes, mehr oder weniger objektorientiertes API verpackt. Der Sourcecode dazu steht in der Datei `vgg16.py`.

In [ ]:
# As large as you can, but no larger than 64 is recommended. 
# If you have an older or cheaper GPU, you'll run out of memory, so will have to decrease this.
batch_size=64

# Wie viele Durchläufe durch die Trainingsdaten sollen gemacht werden:
epochs = 1

# Import VGG16 class, and instantiate
import vgg16; reload(vgg16)
from vgg16 import Vgg16
vgg = Vgg16()

Wir laden die Trainings- und Validierungsdaten als "Batches". `vgg.get_batches()` setzt voraus, dass die Daten in Unterverzeichnissen je Kategorie abgelegt sind. Genau das ist bei uns der Fall, wie wir oben gesehen haben.

Dann wird das VGG16 Modell an unsere Aufgabe ("cat or dog" Klassifizierung) angepasst: `vgg.finetune()`.

In [ ]:
batches = vgg.get_batches(os.path.join(path,'train'), batch_size=batch_size)
val_batches = vgg.get_batches(os.path.join(path,'valid'), batch_size=batch_size*2)
vgg.finetune(batches)

In [ ]:
??vgg.finetune

In [ ]:
??vgg.ft

Jetzt trainieren wir das Modell mit den Daten über `vgg.fit()`. Dabei wird in Wahrheit nur noch der letze (modifizierte) Layer des angepassten VGG16 Modells trainiert.

Die angepassten Gewichte des Modells schreiben wir in eine Datei, so dass wir sie später wieder laden können und so nicht jedesmal das Training wiederholen müssen.

In [ ]:
# Learning rate:
vgg.model.optimizer.lr = 0.01

results_path = os.path.join(path,'results')

for epoch in range(1,epochs+1):
    print 'fit epoch {}'.format(epoch)
    vgg.fit(batches, val_batches, nb_epoch=1)
    weights_filename = os.path.join(results_path,'vgg_weights-{}.h5'.format(epoch))
    print 'saving weights to {}'.format(weights_filename)
    vgg.model.save_weights(weights_filename)


## Vorhersage / Klassifizierung (Prediction)

Jetzt klassifizieren wir die Bilder, die im Unterverzeichnis 'test' abgelegt sind.

Prediction Setup:

In [ ]:
test_path = os.path.join(path,'test')
predictions_file = os.path.join(path,'results/predictions.dat')
filenames_file = os.path.join(path,'results/filenames.dat')

import utils
import time

Prediction durchführen und Ergebnisse speichern:

In [ ]:
print('start predicting at {}'.format(time.asctime()))
batches, predictions = vgg.test(test_path,batch_size=batch_size*2)
print('stop predicting at {}'.format(time.asctime()))

utils.save_array(predictions_file, predictions)
utils.save_array(filenames_file, batches.filenames)

## Mal ein paar Ergebnisse anschauen....

Wir wählen zufällig ein paar Bilder aus und zeigen sie mit der Vorhersage an (`[Wahrscheinlichkeit Katze, Wahrscheinlichkeit Hund]`).

In [ ]:
predictions = utils.load_array(predictions_file)
filenames = utils.load_array(filenames_file)

In [ ]:
from keras.preprocessing import image
def plots_idx(idx, path, filenames, titles=None):
    utils.plots([image.load_img(os.path.join(path,filenames[i])) for i in idx], titles=titles)

In [ ]:
idx = np.random.randint(0, len(batches.filenames),4)
plots_idx(idx, path=path+'test/', filenames=batches.filenames, titles=predictions[idx])

## Visualisieren

Wir wollen uns anschauen, wie gut unser Modell eigentlich vorhersagt. Die Idee dazu ist, dass wir mit dem Modell eine Vorhersage über die bereits klassifizierten Trainingsdaten machen. So kennen wir die "ground truth" zu jedem Bild und können ermitteln, ob die Vorhersage korrekt war.

In [ ]:
if not 'path' in locals():
    path = "sample/"
    # path = "data/"

#import numpy as np
import utils; reload(utils)
#import vgg16; reload(vgg16)
#from vgg16 import Vgg16

Die trainierten Gewichte werden in das Modell geladen:

In [ ]:
vgg = Vgg16()

weights_filename = '{path}results/vgg_weights-{epoch}.h5'.format(path=path,epoch=1)
vgg.model.load_weights(weights_filename)

Vorhersage mit den Validierungsdaten. So kennnen wir die "ground truth" und können sie mit der Vorhersage des Modells vergleichen.

In [ ]:
valid_batches, predictions = vgg.test(path+'valid', batch_size=64)
expected_labels = valid_batches.classes
filenames = valid_batches.filenames

print predictions[:8]

our_predictions = predictions[:,0]
our_labels = np.round(1-our_predictions)
print our_labels[:8]

### Zeige einige korrekte Klassifizierungen

In [ ]:
correct = np.where(our_labels==expected_labels)[0]
print "Found {} correct labels".format(len(correct))
idx = np.random.permutation(correct)[:4]
plots_idx(idx, path+'valid', filenames, our_labels[idx])

### Zeige einige falsche Klassifizierungen

In [ ]:
incorrect = np.where(our_labels!=expected_labels)[0]
print "Found {} incorrect labels.".format(len(incorrect))
idx = np.random.permutation(incorrect)[:4]
plots_idx(idx, path+'valid', filenames, our_labels[idx])

### Zeige einige richtige Klassifizierungen mit der größten Wahrscheintlichkeit

... also da, wo das Modell wirklich recht hatte.

In [ ]:
confident_cats = np.where((our_labels==expected_labels) & (our_labels==0))[0]
print "Found {} confident correct cats labels".format(len(confident_cats))
idx = np.random.permutation(confident_cats)[:4]
plots_idx(idx,path+'valid', filenames, our_labels[idx])

confident_dogs = np.where((our_labels==expected_labels) & (our_labels==1))[0]
print "Found {} confident correct dogs labels".format(len(confident_dogs))
idx = np.random.permutation(confident_dogs)[:4]
plots_idx(idx,path+'valid', filenames, our_labels[idx])

### Ziege einige falsche Klassifizierungen mit der größten Wahrscheinlichkeit

... also die, bei denen das Modell total daneben lag.

In [ ]:
confident_cats = np.where((our_labels!=expected_labels) & (our_labels==0))[0]
print "Found {} confident incorrect cats labels".format(len(confident_cats))
if len(confident_cats)>0:
    idx = np.random.permutation(confident_cats)[:4]
    plots_idx(idx,path+'valid', filenames, our_labels[idx])

confident_dogs = np.where((our_labels!=expected_labels) & (our_labels==1))[0]
print "Found {} confident incorrect dogs labels".format(len(confident_dogs))
if len(confident_dogs)>0:
    idx = np.random.permutation(confident_dogs)[:4]
    plots_idx(idx,path+'valid', filenames, our_labels[idx])

### Ziege die unsichersten Klassifizierungen

... also die, bei denen sich das Modell nicht so sicher war.

In [ ]:
uncertain = np.argsort(np.abs(our_predictions-0.5))
plots_idx(uncertain[:6],path+'valid', filenames, our_predictions[uncertain])

### Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(expected_labels,our_labels)
utils.plot_confusion_matrix(cm, valid_batches.class_indices)